In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

importlib.reload(utils)
importlib.reload(rt)

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
air_joined = pd.read_parquet('airtable_joined.parquet')

In [3]:
analysis_date = dt.date(2022, 2, 8) ##tuesday, new tables

In [4]:
colorscale = branca.colormap.step.RdYlGn_10.scale(vmin=0, vmax=30)
colorscale.caption = "Speed (miles per hour)"

In [5]:
# d3 = [273, 372, 221]

In [6]:
ran_operators = [int(filename.split('_')[0]) for filename in os.listdir('speedmaps') if filename[0] != '.']

In [7]:
pbar = tqdm()

0it [00:00, ?it/s]

In [8]:
not_ran = [380, 343, 360, 159, 346]

In [9]:
# for agency in air_joined.calitp_itp_id.unique():
for agency in not_ran:
    not_ran_operators = []
    # if agency in ran_operators:
    #     print(f'already ran: {agency}')
    #     continue
    print(f'calculating for agency: {agency}...')
    try:
        rt_day = rt.OperatorDayAnalysis(agency, analysis_date, pbar)
        rt_day.set_filter(start_time='15:00', end_time='19:00')
        rt_day.filter_formatted = ', PM Peak, Feb 08 (Tue)' ##TODO fix hardcode
        rt_day.agency_name = (air_joined >> filter(_.calitp_itp_id == agency)).Name.iloc[0]
        m = rt_day.segment_speed_map(how='low_speeds', colorscale=colorscale, size = [1300, 700])
        m.save(f'./speedmaps/{agency}_02_08_pm_peak.html')
        print(f'complete for agency: {agency}')
    except Exception as e:
        print(f'rt failed for agency {agency}')
        not_ran_operators += [agency]
        print(e)

calculating for agency: 380...
found parquet
found parquet
found parquet
found parquet
rt failed for agency 380
vehicle positions trip ids not in schedule
calculating for agency: 343...
found parquet
found parquet
found parquet
found parquet
rt failed for agency 343
vehicle positions trip ids not in schedule
calculating for agency: 360...
found parquet
found parquet
found parquet
found parquet
rt failed for agency 360
vehicle positions trip ids not in schedule
calculating for agency: 159...
found parquet
found parquet
found parquet
found parquet
found_parquet
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
vehicle positions gdf must not be empty
could not generate delays for trip t_1057283_b_25985_tn_0
strptime() argument 1 must be str, not None
could not generate delays for trip t_1057280_b_25985_tn_0
strptime() argument 1 must be str, not None
could not generate delays for trip t

In [8]:
# air_joined['url'] = air_joined.apply(lambda x:
#                     f'http://docs.calitp.org/data-analyses/rt_delay/speedmaps/{x.calitp_itp_id}_02_08_pm_peak.html',
#                                     axis = 1)

# air_joined.to_csv('linked.csv')